# Prohledávání hyperparametrů pro model MobileNetV2 nad augmentovaným datasetem CIFAR100

Tento notebook slouží k nalezení optimálních hyperparametrů nad augmentovaným datasetem CIFAR100 pro model MobileNetV2. Hyperparametry jsou hledány pro všechny varianty modelu (náhodně inicializovaný, předtrénovaný (doučení klasifikační hlavy) a předtrénovaný (doučení celého modelu)). Pro každou z variant jsou hledány hyperparametry pro normální trénink a trénink s destilací. 

K prohledávání je využito knihovny Optuna s algoritmem Hyperband. Nejlepší konfigurace je volena na základě F1-skóre, zkoušeno je 150 kombinací hyperparametrů pro každou variantu modelu. 

## Import knihoven a základní nastavení

In [1]:
from transformers import Trainer
from torch.utils.data import ConcatDataset
import optuna
import torch
import math
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [ ]:
dataset_part = base.get_dataset_part()

Resetování náhodného seedu pro replikovatelnost výsledků.

In [3]:
base.reset_seed()

Ověření dostupnosti GPU.

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA H100 PCIe


Načtení datasetu a aplikace základních transformací.

In [5]:
DATASET = "cifar100"

In [6]:
transform = base.base_transforms()

train = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=transform, device="cpu")
eval = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.EVAL, transform=transform, device="cpu")
test = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TEST, transform=transform, device="cpu")

Provedení filtrace augmentovaného datasetu dle popsaného mechanismu.

In [7]:
augment_transform = base.aug_transforms()

train_aug = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=augment_transform)
train_aug = base.remove_diff_pred_class(train, train_aug, pytorch_dataset=True)
train_combo = ConcatDataset([train, train_aug])

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

Základní konfigurace tréninku během prohledávání. Optuna nepracuje s epochami, ale s kroky. Níže je prováděn přepočet. 

Minimální délka tréninku jsou tři epochy, maximální deset epoch. Maximální počet kroků pro warm up je nastaven na 10 % první epochy.

In [9]:
num_epochs = 10
batch_size = 128

In [10]:
data_length = len(train)
min_r = math.ceil(data_length/batch_size)*3
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

## Prohledávání s normálním tréninkem náhodně inicializovaného modelu
Definice hledaných hyperparametrů a jejich rozmezí.

In [11]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [12]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [13]:
base.reset_seed()

Konfigurace jednotlivých tréninků.

In [35]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/random_hp-search", logging_dir=f"~/logs/{DATASET}/random_hp-search", epochs=num_epochs, batch_size=batch_size)

Konfigurace trenéra pro jednotlivé tréninky. 

In [36]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_random_init_mobilenet(100)
)
  

Nastavení prohledávání.

In [ ]:
best_base_random = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-random",
    n_trials=150
)

[I 2025-04-03 22:34:22,000] A new study created in memory with name: Base-random


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,4.050400,3.384866,0.165600,0.152353,0.165600,0.126850
2,3.210900,2.734753,0.289700,0.284892,0.289700,0.269228


[I 2025-04-03 22:37:18,915] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 5}


...

Trial 144 with params: {'learning_rate': 0.0003612235396496027, 'weight_decay': 0.001, 'warmup_steps': 22}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,4.029000,3.393000,0.159900,0.153837,0.159900,0.124344
2,3.204100,2.722409,0.294700,0.294734,0.294700,0.274855
3,2.669800,2.300364,0.383400,0.394763,0.383400,0.369269
4,2.293800,2.083744,0.435400,0.442049,0.435400,0.422239


[I 2025-04-04 19:19:18,804] Trial 144 pruned. 


Trial 145 with params: {'learning_rate': 0.00040788796347888587, 'weight_decay': 0.01, 'warmup_steps': 21}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,4.032400,3.351868,0.167800,0.145844,0.167800,0.134047
2,3.191100,2.732506,0.284400,0.290117,0.284400,0.264913
3,2.670500,2.342363,0.370100,0.377923,0.370100,0.354937
4,2.302900,2.108405,0.429400,0.427022,0.429400,0.415173
5,2.010000,1.940041,0.467100,0.467633,0.467100,0.453245


In [ ]:
print(best_base_random)

In [14]:
base.reset_seed()

## Prohledávání s destilací náhodně inicializovaného modelu

In [15]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/random-KD_hp-search", logging_dir=f"~/logs/{DATASET}/random-KD_hp-search",  remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.

In [16]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [17]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [18]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_random_init_mobilenet(100)
)
  

Nastavení prohledávání.

In [19]:
best_distill_random = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-04-04 19:44:27,010] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.698600,3.187188,0.189300,0.175951,0.189300,0.145527
2,3.009700,2.654963,0.317100,0.326876,0.317100,0.294212


[I 2025-04-04 19:47:09,967] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 28, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.673500,3.234220,0.173200,0.161848,0.173200,0.131267
2,3.028800,2.706821,0.307400,0.319275,0.307400,0.281637
3,2.593100,2.318923,0.392400,0.409540,0.392400,0.374209
4,2.275500,2.098227,0.454800,0.459560,0.454800,0.437940


[I 2025-04-05 15:51:56,180] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.000397489159098293, 'weight_decay': 0.0, 'warmup_steps': 3, 'lambda_param': 0.9, 'temperature': 4.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.662400,3.208494,0.176400,0.174639,0.176400,0.134337
2,3.001900,2.666571,0.314100,0.314493,0.314100,0.288047
3,2.559500,2.285873,0.409400,0.425981,0.409400,0.393328
4,2.242800,2.069414,0.460500,0.467822,0.460500,0.445006
5,1.985800,1.933515,0.487400,0.482714,0.487400,0.470192
6,1.775000,1.857854,0.512600,0.526089,0.512600,0.505382
7,1.593400,1.745511,0.533400,0.536616,0.533400,0.524154
8,1.426000,1.698594,0.552400,0.560179,0.552400,0.548677
9,1.286000,1.656166,0.559400,0.564904,0.559400,0.554819
10,1.174000,1.667190,0.556300,0.562955,0.556300,0.553087


[I 2025-04-05 16:06:23,363] Trial 149 finished with value: 0.5530867062854452 and parameters: {'learning_rate': 0.000397489159098293, 'weight_decay': 0.0, 'warmup_steps': 3, 'lambda_param': 0.9, 'temperature': 4.5}. Best is trial 78 with value: 0.5691301587796576.


In [20]:
print(best_distill_random)

BestRun(run_id='78', objective=0.5691301587796576, hyperparameters={'learning_rate': 0.0005863729932744796, 'weight_decay': 0.004, 'warmup_steps': 7, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}, run_summary=None)


In [21]:
base.reset_seed()

## Prohledávání s normálním tréninkem s doučením klasifikační hlavy předtrénovaného modelu
Konfigurace jednotlivých tréninků.

In [22]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained-head_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained-head_hp-search", epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí.

In [23]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [24]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace trenéra pro jednotlivé tréninky. 

In [25]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.freeze_model(base.get_mobilenet(100))
)
  

Nastavení prohledávání.

In [26]:
best_base_head = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-04-05 18:38:48,172] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.370300,2.430507,0.469900,0.482332,0.469900,0.455055
2,2.382800,2.012310,0.509400,0.519077,0.509400,0.504219


[I 2025-04-05 18:41:25,909] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.598400,1.872946,0.516900,0.548535,0.516900,0.509881
2,1.954500,1.742102,0.536000,0.560894,0.536000,0.534916
3,1.814500,1.707214,0.537800,0.554071,0.537800,0.534473
4,1.741900,1.679145,0.548400,0.564194,0.548400,0.544165
5,1.685100,1.689675,0.543300,0.552345,0.543300,0.535558
6,1.644000,1.673656,0.550100,0.562046,0.550100,0.546697
7,1.610200,1.654312,0.555100,0.556733,0.555100,0.548578
8,1.581000,1.645759,0.556800,0.563430,0.556800,0.554627


[I 2025-04-06 14:50:16,288] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0006931015834947695, 'weight_decay': 0.006, 'warmup_steps': 26}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.827500,1.970860,0.508300,0.528053,0.508300,0.500596
2,2.036300,1.774988,0.535800,0.553391,0.535800,0.534527
3,1.875100,1.726127,0.538200,0.550719,0.538200,0.534196
4,1.794200,1.687042,0.547800,0.559656,0.547800,0.544088
5,1.735500,1.690608,0.544900,0.548815,0.544900,0.537550
6,1.695100,1.672207,0.552700,0.559728,0.552700,0.548505
7,1.664200,1.655625,0.555400,0.555574,0.555400,0.548905
8,1.638100,1.649549,0.558400,0.563268,0.558400,0.555796
9,1.620300,1.632837,0.559200,0.555835,0.559200,0.553361
10,1.601800,1.642009,0.556800,0.556152,0.556800,0.552872


[I 2025-04-06 15:03:47,471] Trial 149 finished with value: 0.5528721344486837 and parameters: {'learning_rate': 0.0006931015834947695, 'weight_decay': 0.006, 'warmup_steps': 26}. Best is trial 23 with value: 0.5563036082256244.


In [27]:
print(best_base_head)

BestRun(run_id='23', objective=0.5563036082256244, hyperparameters={'learning_rate': 0.0010450619114319906, 'weight_decay': 0.005, 'warmup_steps': 9}, run_summary=None)


In [28]:
base.reset_seed()

## Prohledávání s destilací s doučením klasifikační hlavy předtrénovaného modelu
Konfigurace jednotlivých tréninků.

In [29]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained-head-KD_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained-head-KD_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.


In [30]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [31]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [32]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.freeze_model(base.get_mobilenet(100))
)

Nastavení prohledávání.

In [33]:
best_distill_head = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-04-06 15:03:48,219] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.184300,2.575029,0.464800,0.482607,0.464800,0.448079
2,2.513100,2.288072,0.509400,0.523166,0.509400,0.500863


[I 2025-04-06 15:06:24,659] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 28, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.616100,2.186422,0.520300,0.551539,0.520300,0.513947
2,2.328500,2.152302,0.525700,0.560239,0.525700,0.522100


[I 2025-04-07 10:18:10,444] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0010030210472888152, 'weight_decay': 0.007, 'warmup_steps': 4, 'lambda_param': 1.0, 'temperature': 2.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.690500,2.214449,0.515600,0.539547,0.515600,0.506771
2,2.287500,2.107025,0.534400,0.553682,0.534400,0.528753
3,2.212700,2.081971,0.536500,0.552779,0.536500,0.530499
4,2.178000,2.051279,0.544900,0.556865,0.544900,0.539446
5,2.148600,2.059483,0.543000,0.546276,0.543000,0.531251
6,2.128900,2.041957,0.550500,0.559134,0.550500,0.543940
7,2.111500,2.024155,0.555900,0.556505,0.555900,0.547865
8,2.097700,2.014355,0.556300,0.561970,0.556300,0.551714
9,2.084000,2.006403,0.557600,0.556469,0.557600,0.550339
10,2.068300,2.012084,0.554500,0.553810,0.554500,0.548984


[I 2025-04-07 10:31:33,872] Trial 149 finished with value: 0.5489836397092521 and parameters: {'learning_rate': 0.0010030210472888152, 'weight_decay': 0.007, 'warmup_steps': 4, 'lambda_param': 1.0, 'temperature': 2.0}. Best is trial 91 with value: 0.550545336158113.


In [34]:
print(best_distill_head)

BestRun(run_id='91', objective=0.550545336158113, hyperparameters={'learning_rate': 0.0012457733887986462, 'weight_decay': 0.001, 'warmup_steps': 10, 'lambda_param': 0.2, 'temperature': 5.0}, run_summary=None)


In [11]:
base.reset_seed()

## Prohledávání s normálním tréninkem s doučením předtrénovaného modelu
Konfigurace jednotlivých tréninků.

In [12]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained_hp-search", epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí.

In [13]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.


In [14]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace trenéra pro jednotlivé tréninky. 

In [15]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_mobilenet(100)
)
  


Nastavení prohledávání.


In [ ]:
best_base_pretrained = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-04-01 22:44:31,611] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.626400,0.932228,0.722900,0.730839,0.722900,0.721594
2,0.666200,0.846432,0.749500,0.759946,0.749500,0.748810


[I 2025-04-01 22:47:17,504] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.531000,0.965550,0.718100,0.730890,0.718100,0.717623
2,0.668300,0.870007,0.748500,0.758453,0.748500,0.747329
3,0.380100,0.856204,0.760800,0.773363,0.760800,0.760384
4,0.216500,0.905064,0.756800,0.771629,0.756800,0.758103


[I 2025-04-02 18:38:44,950] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0004336974186584821, 'weight_decay': 0.008, 'warmup_steps': 3}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.472000,0.991572,0.707100,0.720574,0.707100,0.705228
2,0.675200,0.888380,0.741200,0.754081,0.741200,0.740856
3,0.393700,0.871773,0.757200,0.770145,0.757200,0.756088
4,0.233500,0.899380,0.761600,0.772784,0.761600,0.762336
5,0.132100,0.944402,0.764600,0.775747,0.764600,0.764671
6,0.073700,0.995261,0.763100,0.771544,0.763100,0.763176
7,0.042300,0.987470,0.770500,0.775231,0.770500,0.768964
8,0.020600,1.016830,0.776600,0.782365,0.776600,0.775963


[I 2025-04-02 18:49:56,159] Trial 149 pruned. 


In [ ]:
print(best_base_pretrained)

BestRun(run_id='8', objective=0.7885223960150077, hyperparameters={'learning_rate': 0.00040842279473800845, 'weight_decay': 0.008, 'warmup_steps': 6}, run_summary=None)


In [ ]:
base.reset_seed()


## Prohledávání s destilací s doučením předtrénovaného modelu
Konfigurace jednotlivých tréninků.


In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained-KD_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained-KD_hp-search",  remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.

In [ ]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [ ]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky.

In [ ]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_mobilenet(100)
)

Nastavení prohledávání.

In [ ]:
best_distil_pretrained = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-04-02 19:45:37,252] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.653600,1.035308,0.726800,0.735513,0.726800,0.725206
2,0.807200,0.889646,0.759200,0.769321,0.759200,0.758276


[I 2025-04-02 19:48:27,503] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 28, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.585300,1.194386,0.684300,0.698541,0.684300,0.680506
2,0.938000,1.030606,0.727600,0.742708,0.727600,0.724350


[I 2025-04-03 15:22:05,389] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0003177982910013901, 'weight_decay': 0.008, 'warmup_steps': 12, 'lambda_param': 0.2, 'temperature': 7.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.588500,1.024475,0.727500,0.736608,0.727500,0.725994
2,0.795300,0.897548,0.756900,0.766110,0.756900,0.755684
3,0.543500,0.861646,0.764400,0.779101,0.764400,0.764590
4,0.400500,0.829411,0.774300,0.787318,0.774300,0.775959
5,0.307800,0.839609,0.774200,0.787755,0.774200,0.774570
6,0.246600,0.846964,0.770400,0.783229,0.770400,0.771870
7,0.206800,0.801338,0.784400,0.790644,0.784400,0.784117
8,0.179500,0.788344,0.784300,0.793015,0.784300,0.784733


[I 2025-04-03 15:33:20,794] Trial 149 pruned. 


In [ ]:
print(best_distil_pretrained)

In [ ]:
print("Best random init training score: ", best_base_random)
print("Best random init distilation trianing score: ", best_distill_random)
print("Best pretrained (head only) training score: ", best_base_head)
print("Best pretrained distilation (head only) training score: ",best_distill_head)
print("Best pretrained training score: ", best_base_pretrained)
print("Best pretrained distilation training score: ", best_distil_pretrained)